# CNN vs. ConvNN vs. ConvNN location Accuracy + Speed Test
- Updated implementation: add location channels -> unshuffle -> flatten -> conv -> unflatten -> shuffle -> remove location channels 

In [1]:
# Torch
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch import optim 


# Train + Data  
import sys 
sys.path.append('../Layers')
from Conv1d_NN import *
from Conv2d_NN import *

from Conv1d_NN_spatial import * 
from Conv2d_NN_spatial import * 

sys.path.append('../Data')
from CIFAR10 import * 


sys.path.append('../Train')
from train2d import * 


/Users/mingikang/miniforge3/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## I. Models

In [2]:
# Classic CNN 2D Model - Control Model 

CNN_2D = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),

    nn.Flatten(),
    nn.Linear(32768, 1024),
    nn.ReLU(),
    nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary 
summary(CNN_2D, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
            Conv2d-3           [-1, 32, 32, 32]           4,640
              ReLU-4           [-1, 32, 32, 32]               0
           Flatten-5                [-1, 32768]               0
            Linear-6                 [-1, 1024]      33,555,456
              ReLU-7                 [-1, 1024]               0
            Linear-8                   [-1, 10]          10,250
Total params: 33,570,794
Trainable params: 33,570,794
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.02
Params size (MB): 128.06
Estimated Total Size (MB): 129.09
----------------------------------------------------------------


In [3]:
# ConNN 2D All Sample Model with K = 9
Conv2dNN_9_all = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_all, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 12, 256]               0
            Conv1d-2              [-1, 64, 256]           6,976
         Conv1d_NN-3              [-1, 64, 256]               0
         Conv2d_NN-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 64, 256]               0
            Conv1d-7             [-1, 128, 256]          73,856
         Conv1d_NN-8             [-1, 128, 256]               0
         Conv2d_NN-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [4]:
# ConNN 2D Random Sample n = 64 Model with K = 9
Conv2dNN_9_64 = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=64, 
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=64, 
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_64, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 12, 256]               0
            Conv1d-2              [-1, 64, 256]           6,976
         Conv1d_NN-3              [-1, 64, 256]               0
         Conv2d_NN-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 64, 256]               0
            Conv1d-7             [-1, 128, 256]          73,856
         Conv1d_NN-8             [-1, 128, 256]               0
         Conv2d_NN-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [5]:
# ConNN 2D Spatial Sample n = 8 Model with K = 9
Conv2dNN_spatial_9_8 = nn.Sequential(
   Conv2d_NN_spatial(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=8, 
   ), 
   nn.ReLU(),

   Conv2d_NN_spatial(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=8, 
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_spatial_9_8, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 12, 256]               0
            Conv1d-2              [-1, 64, 256]           6,976
 Conv1d_NN_spatial-3              [-1, 64, 256]               0
 Conv2d_NN_spatial-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 64, 256]               0
            Conv1d-7             [-1, 128, 256]          73,856
 Conv1d_NN_spatial-8             [-1, 128, 256]               0
 Conv2d_NN_spatial-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [6]:
# ConNN 2D All Sample Model with K = 9 with Location Channels
Conv2dNN_9_all_location = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
      location_channels = True
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
      location_channels = True

   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_all_location, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 20, 256]               0
            Conv1d-2              [-1, 72, 256]          13,032
         Conv1d_NN-3              [-1, 72, 256]               0
         Conv2d_NN-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 72, 256]               0
            Conv1d-7             [-1, 136, 256]          88,264
         Conv1d_NN-8             [-1, 136, 256]               0
         Conv2d_NN-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [7]:
# ConNN 2D Random Sample n = 64 Model with K = 9 and Location Channels
Conv2dNN_9_64_location = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=64, 
      location_channels = True
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=64, 
      location_channels = True
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_64_location, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 20, 256]               0
            Conv1d-2              [-1, 72, 256]          13,032
         Conv1d_NN-3              [-1, 72, 256]               0
         Conv2d_NN-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 72, 256]               0
            Conv1d-7             [-1, 136, 256]          88,264
         Conv1d_NN-8             [-1, 136, 256]               0
         Conv2d_NN-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [8]:
# ConNN 2D Spatial Sample n = 8 Model with K = 9 and Location Channels
Conv2dNN_spatial_9_8_location = nn.Sequential(
   Conv2d_NN_spatial(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=8, 
      location_channels = True
   ), 
   nn.ReLU(),

   Conv2d_NN_spatial(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples=8, 
      location_channels = True
   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_spatial_9_8_location, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 20, 256]               0
            Conv1d-2              [-1, 72, 256]          13,032
 Conv1d_NN_spatial-3              [-1, 72, 256]               0
 Conv2d_NN_spatial-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 72, 256]               0
            Conv1d-7             [-1, 136, 256]          88,264
 Conv1d_NN_spatial-8             [-1, 136, 256]               0
 Conv2d_NN_spatial-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

## II. Training

In [9]:
# CIFAR10
cifar10 = CIFAR10()

Files already downloaded and verified
Files already downloaded and verified


In [10]:
# Classic CNN 2D Model - Control Model 
CNN_2D.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_2D.parameters(), lr=0.001)
num_epochs = 10 
train_model(CNN_2D, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(CNN_2D, cifar10.test_loader)

Epoch 1, Time: 20.58729910850525, Loss: 1.3464747151297987
Epoch 2, Time: 20.23523712158203, Loss: 0.81139599091714
Epoch 3, Time: 19.836535215377808, Loss: 0.3958661693250737
Epoch 4, Time: 19.92248797416687, Loss: 0.10977358032789682
Epoch 5, Time: 19.375409841537476, Loss: 0.053613974532955674
Epoch 6, Time: 19.44871187210083, Loss: 0.05030135700778078
Epoch 7, Time: 19.722390174865723, Loss: 0.05096490231826139
Epoch 8, Time: 19.73980212211609, Loss: 0.038862013319671596
Epoch 9, Time: 19.751009225845337, Loss: 0.03369374922104537
Epoch 10, Time: 19.708847999572754, Loss: 0.038186812118547936

 Average epoch time: 19.832773065567018
Accuracy on test set: 63.62%


63.62

In [11]:
# ConNN 2D All Sample Model with K = 9
Conv2dNN_9_all.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_all.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_all, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_all, cifar10.test_loader)

Epoch 1, Time: 54.05575084686279, Loss: 1.582684606085043
Epoch 2, Time: 53.622182846069336, Loss: 1.2448970063415634
Epoch 3, Time: 53.538856983184814, Loss: 1.0540408575931168
Epoch 4, Time: 53.50482726097107, Loss: 0.8297277970234757
Epoch 5, Time: 53.31831884384155, Loss: 0.6024696168768436
Epoch 6, Time: 52.96550393104553, Loss: 0.40016375390617437
Epoch 7, Time: 53.174283027648926, Loss: 0.26924835905299316
Epoch 8, Time: 54.47589731216431, Loss: 0.2253233072207407
Epoch 9, Time: 54.0193727016449, Loss: 0.16799291739683322
Epoch 10, Time: 52.61531901359558, Loss: 0.15747264950343257

 Average epoch time: 53.52903127670288
Accuracy on test set: 54.18%


54.18

In [12]:
# ConNN 2D Random Sample n = 64 Model with K = 9
Conv2dNN_9_64.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_64.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_64, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_64, cifar10.test_loader)

Epoch 1, Time: 56.46987009048462, Loss: 1.720914816155153
Epoch 2, Time: 55.64385795593262, Loss: 1.3699956707027563
Epoch 3, Time: 56.707412004470825, Loss: 1.219037377925785
Epoch 4, Time: 57.115807056427, Loss: 1.079085472828287
Epoch 5, Time: 55.92519497871399, Loss: 0.9322226866889183
Epoch 6, Time: 55.3672559261322, Loss: 0.7811099042749161
Epoch 7, Time: 57.02142810821533, Loss: 0.6268117588270655
Epoch 8, Time: 56.82557797431946, Loss: 0.49788993726605957
Epoch 9, Time: 55.25615429878235, Loss: 0.39125385899525467
Epoch 10, Time: 55.6535439491272, Loss: 0.3313450827680128

 Average epoch time: 56.198610234260556
Accuracy on test set: 53.52%


53.52

In [13]:
# ConNN 2D Spatial Sample n = 8 => n^2 = 64 Model with K = 9
Conv2dNN_spatial_9_8.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_spatial_9_8.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_spatial_9_8, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_spatial_9_8, cifar10.test_loader)

Epoch 1, Time: 55.89957594871521, Loss: 1.6452178959651371
Epoch 2, Time: 54.69520807266235, Loss: 1.3316000028491934
Epoch 3, Time: 55.65709590911865, Loss: 1.1406195205648233
Epoch 4, Time: 57.16094374656677, Loss: 0.960253652556778
Epoch 5, Time: 54.536158084869385, Loss: 0.7690938668863853
Epoch 6, Time: 54.42297101020813, Loss: 0.5785821724654464
Epoch 7, Time: 54.34545612335205, Loss: 0.40732229059881264
Epoch 8, Time: 55.149595975875854, Loss: 0.29635062513639554
Epoch 9, Time: 55.624682903289795, Loss: 0.23647923063953666
Epoch 10, Time: 55.988527059555054, Loss: 0.18414864026229172

 Average epoch time: 55.34802148342133
Accuracy on test set: 52.12%


52.12

### Location Channels added


In [16]:
# ConNN 2D All Sample Model with K = 9 with Location Channels
Conv2dNN_9_all_location.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_all_location.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_all_location, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_all_location, cifar10.test_loader)

Epoch 1, Time: 124.02667307853699, Loss: 0.4552967606488701
Epoch 2, Time: 141.48303890228271, Loss: 0.34518935645708954
Epoch 3, Time: 150.06747913360596, Loss: 0.26144137765611986
Epoch 4, Time: 141.14968299865723, Loss: 0.21994776444990768
Epoch 5, Time: 142.4051058292389, Loss: 0.18218493657400997
Epoch 6, Time: 141.56225609779358, Loss: 0.17245024487690624
Epoch 7, Time: 142.38091278076172, Loss: 0.15440105963641268
Epoch 8, Time: 141.77669882774353, Loss: 0.13778662604644246
Epoch 9, Time: 128.69672513008118, Loss: 0.12897813161406332
Epoch 10, Time: 147.67101287841797, Loss: 0.10797681266744442

 Average epoch time: 140.12195856571196
Accuracy on test set: 53.75%


53.75

In [18]:
# ConNN 2D Random Sample n = 64 Model with K = 9 with Location Channels
Conv2dNN_9_64_location.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_64_location.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_64_location, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_64_location, cifar10.test_loader)

Epoch 1, Time: 114.20284724235535, Loss: 1.3818144160310935
Epoch 2, Time: 155.6986391544342, Loss: 1.321403600904338
Epoch 3, Time: 125.91427898406982, Loss: 1.2706110371500634
Epoch 4, Time: 141.98924207687378, Loss: 1.2237575779028256
Epoch 5, Time: 141.60129833221436, Loss: 1.1736151797082417
Epoch 6, Time: 144.2935779094696, Loss: 1.1294377061259715
Epoch 7, Time: 151.37798476219177, Loss: 1.084236878644475
Epoch 8, Time: 166.663649559021, Loss: 1.0296185437370748
Epoch 9, Time: 145.22230696678162, Loss: 0.9857760042790562
Epoch 10, Time: 150.49387216567993, Loss: 0.9411968805296037

 Average epoch time: 143.74576971530914
Accuracy on test set: 52.68%


52.68

In [15]:
# ConNN 2D Spatial Sample n = 8 => n^2 = 64 Model with K = 9 with Location Channels
Conv2dNN_spatial_9_8_location.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_spatial_9_8_location.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_spatial_9_8_location, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_spatial_9_8_location, cifar10.test_loader)

Epoch 1, Time: 103.213693857193, Loss: 1.6696802650571174
Epoch 2, Time: 143.6284213066101, Loss: 1.412518604332224
Epoch 3, Time: 148.1703839302063, Loss: 1.2791596375158072
Epoch 4, Time: 152.92733001708984, Loss: 1.1507666710849918
Epoch 5, Time: 149.7355740070343, Loss: 1.0203461579197204
Epoch 6, Time: 140.24775099754333, Loss: 0.8963169542038837
Epoch 7, Time: 149.68256521224976, Loss: 0.7657066256646305
Epoch 8, Time: 147.88176107406616, Loss: 0.63381688010967
Epoch 9, Time: 148.95815992355347, Loss: 0.5212805496762171
Epoch 10, Time: 148.16730070114136, Loss: 0.4329154521341214

 Average epoch time: 143.26129410266876
Accuracy on test set: 52.06%


52.06